In [ ]:
!pip install sentence_transformers
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=df0210dbc85572aa2984348de8d777ff47b5b5c52e465e0b69bbd833591b9499
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install pyngrok


In [ ]:
!pip install flask_ngrok

In [ ]:
!ngrok authtoken 2n16p7O9KiaWI37VPdO7g4xwA85_7LPPyGxKoQyScjqjkuH6c


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
def call_daily_workout_api(user_id, focus_muscle_list, time_available, langue):
    url = "https://a0ab-102-157-57-191.ngrok-free.app/create-daily-workout-for-time/"
    data = {
        "user_id": user_id,
        "focus_muscle_list": focus_muscle_list,
        "time_available": time_available,
        "langue": langue  # Ajout du paramètre langue
    }
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Lève une exception pour les erreurs HTTP
        return response.json().get("workout_plan")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None


# Appel de l'API avec les données nécessaires
user_id = 1
focus_muscle_list = ['chest', 'back']
time_available = 30
langue = 'french'

response = call_daily_workout_api(user_id, focus_muscle_list, time_available, langue)

print(response)


{'Warm-Up': [{'name': 'Câble croisé Paul Carter', 'muscles_involved': ['poitrine']}], 'Main Workout': [{'name': 'Câble croisé Paul Carter', 'muscles_involved': ['poitrine']}, {'name': 'Presse à banc incliné Paul Carter', 'muscles_involved': ['poitrine']}, {'name': 'Paul Carter - Tirage à la poulie haute', 'muscles_involved': ['milieu du dos']}, {'name': 'Pull PJR-', 'muscles_involved': ['dorsaux']}], 'Cool-Down': [{'name': 'Yoga plex', 'muscles_involved': ['bas du dos']}]}


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
from langdetect import detect
import re
import requests
import webbrowser
import threading
import gc  # Pour gérer manuellement le garbage collector
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

# Charger les modèles et les tokenizers
from transformers import BartForConditionalGeneration, BartTokenizer, MBartForConditionalGeneration, MBart50TokenizerFast
# Initialiser Flask
app = Flask(__name__)
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    bart_model_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_bart_final_10epoch'
    tokenizer_en = BartTokenizer.from_pretrained(bart_model_path)
    model_en = BartForConditionalGeneration.from_pretrained(bart_model_path).to(device)

    mbart_model_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_mbart_final_3epoch'
    tokenizer_fr = MBart50TokenizerFast.from_pretrained(mbart_model_path, src_lang="fr_XX", tgt_lang="fr_XX")
    model_fr = MBartForConditionalGeneration.from_pretrained(mbart_model_path).to(device)
except Exception as e:
    print(f"Erreur lors du chargement des modèles : {e}")

# Charger les datasets
try:
    french_data = pd.read_csv('/content/french_dataset.csv')
    english_data = pd.read_csv('/content/english_dataset.csv')
except FileNotFoundError as e:
    print(f"Erreur de chargement des datasets : {e}")

# Charger les modèles Sentence-BERT multilingue et Cross-Encoder
bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

@app.route('/')
def home():
    # Contenu HTML directement servi ici
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Chatbot Interface</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f1f1f1;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
            }
            .chat-container {
                width: 400px;
                max-width: 100%;
                height: 600px;
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
                overflow: hidden;
                display: flex;
                flex-direction: column;
            }
            .welcome-message {
                background-color: #007bff;
                color: white;
                padding: 15px;
                text-align: center;
                font-weight: bold;
            }
            .chat-box {
                flex: 1;
                overflow-y: auto;
                padding: 20px;
                display: flex;
                flex-direction: column;
            }
            .user-message, .bot-message {
                display: flex;
                margin-bottom: 10px;
            }
            .user-message .message, .bot-message .message {
                padding: 10px;
                border-radius: 20px;
                max-width: 70%;
            }
            .user-message .message {
                background-color: #007bff;
                color: white;
                margin-left: auto;
            }
            .bot-message .message {
                background-color: #f1f1f1;
                color: #000;
                margin-right: auto;
            }
            .input-box {
                display: flex;
                padding: 10px;
                border-top: 1px solid #ccc;
            }
            .input-box input {
                flex: 1;
                padding: 10px;
                border: none;
                border-radius: 20px;
                margin-right: 10px;
            }
            .input-box button {
                padding: 10px 20px;
                background-color: #007bff;
                border: none;
                color: white;
                border-radius: 20px;
                cursor: pointer;
            }
        </style>
    </head>
    <body>
        <div class="chat-container">
            <div class="welcome-message">
              Bonjour et Welcome to Gymie !             </div>

            <div class="chat-box" id="chat-box">
                <!-- Message de bienvenue -->
                <div class="bot-message">
                    <div class="message">
                    Salut ! Je suis Gymie,votre Assistant Personnel de Fitness . 🏋️‍♂️
                    Posez vos questions en français ou en anglais, et je vous aiderai à personnaliser vos entraînements quotidiens ou mensuels. 🚀
                    Commençons ensemble et faisons de chaque jour un pas de plus vers une vie plus saine et plus active ! 💪
                    </div>
                </div>
                <!-- Chat messages will appear here -->
            </div>
            <div class="input-box">
                <input type="text" id="user-input" placeholder="Posez votre question ici..." />
                <button onclick="sendMessage()">Envoyer</button>
            </div>
        </div>
        <script>
            function sendMessage() {
                var userInput = document.getElementById('user-input').value;
                var chatBox = document.getElementById('chat-box');
                // Append user message
                var userMessageDiv = document.createElement('div');
                userMessageDiv.className = 'user-message';
                userMessageDiv.innerHTML = '<div class="message">' + userInput + '</div>';
                chatBox.appendChild(userMessageDiv);
                // Scroll to the bottom of the chat box
                chatBox.scrollTop = chatBox.scrollHeight;
                // Clear input
                document.getElementById('user-input').value = '';
                // Send user input to server
                fetch('/get_response', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ message: userInput }),
                })
                .then(response => response.json())
                .then(data => {
                    // Append bot response
                    var botMessageDiv = document.createElement('div');
                    botMessageDiv.className = 'bot-message';
                    botMessageDiv.innerHTML = '<div class="message">' + data.response + '</div>';
                    chatBox.appendChild(botMessageDiv);
                    // Scroll to the bottom of the chat box
                    chatBox.scrollTop = chatBox.scrollHeight;
                });
            }
        </script>
    </body>
    </html>
    """
    return html_content

@app.route('/get_response', methods=['POST'])
def get_response():
    user_input = request.json.get("message")
    response = chatbot_response(user_input)
    return jsonify({"response": response})

def preprocess_text(text, language):
    text = text.lower()
    if language == 'french':
        text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('à', 'a')
        text = text.replace('ç', 'c').replace('ô', 'o').replace('û', 'u').replace('î', 'i')

    return text

# Nettoyer les données
french_data['Question'] = french_data['Question'].apply(lambda x: preprocess_text(x, 'french'))
english_data['Question'] = english_data['Question'].apply(lambda x: preprocess_text(x, 'english'))

# Créer des listes de questions et réponses
french_questions = french_data['Question'].tolist()
english_questions = english_data['Question'].tolist()
french_responses = french_data['Réponse'].tolist()
english_responses = english_data['Réponse'].tolist()

# Encodage des questions avec le Bi-Encoder
french_embeddings = bi_encoder.encode(french_questions, convert_to_tensor=True)
english_embeddings = bi_encoder.encode(english_questions, convert_to_tensor=True)

def calculate_word_overlap(user_question, best_question):
    user_words = set(user_question.split())
    best_words = set(best_question.split())
    common_words = user_words.intersection(best_words)
    overlap_percentage = len(common_words) / len(user_words) * 100
    return overlap_percentage

# Trouver une question similaire et une réponse
def find_similar_question_and_response(user_question, embeddings, questions, responses, top_k=5, threshold=75):
    user_question_preprocessed = preprocess_text(user_question, detect_language(user_question))
    user_embedding = bi_encoder.encode(user_question_preprocessed, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(user_embedding, embeddings)[0]
    top_k_indices = torch.topk(cosine_scores, k=top_k).indices
    similar_questions = [(questions[idx.item()], responses[idx.item()], cosine_scores[idx].item()) for idx in top_k_indices]

    for question, response, score in similar_questions:
        if score >= 0.95:
            return question, response

    pairs = [[user_question_preprocessed, question] for question, _, _ in similar_questions]
    cross_scores = cross_encoder.predict(pairs)
    best_match_idx = cross_scores.argmax()
    best_question, best_response = similar_questions[best_match_idx][:2]
    word_overlap = calculate_word_overlap(user_question_preprocessed, best_question)

    if word_overlap >= threshold:
        return best_question, best_response
    else:
        return None, None

def detect_language(text):
    try:
        language = detect(text)
        return 'english' if language == 'en' else 'french'
    except:
        return 'french'


def call_daily_workout_api(user_id, focus_muscle_list, time_available, langue):
    url = "https://a0ab-102-157-57-191.ngrok-free.app/create-daily-workout-for-time/"
    data = {
        "user_id": user_id,
        "focus_muscle_list": focus_muscle_list,
        "time_available": time_available,
        "langue": langue  # Ajout du paramètre langue
    }
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Lève une exception pour les erreurs HTTP
        return response.json().get("workout_plan")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None



def call_monthly_workout_api(user_id, langue):
    url = "https://a0ab-102-157-57-191.ngrok-free.app/get_monthly_workout_plan"

    # Préparer les données pour l'API
    data = {
        "user_id": user_id,
        "langue": langue
    }

    # Envoyer la requête POST avec les données JSON
    response = requests.post(url, json=data)

    # Vérifier si la requête a réussi
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()
        return response.json().get("monthly_workout_plan")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return None


# Dictionnaire de traduction des muscles du français vers l'anglais
muscle_translation = {
    'dos': 'back',
    'poitrine': 'chest',
    'core': 'core',
    'epaules': 'shoulders',
    'jambes': 'legs',
    'bras': 'arms',
    'biceps': 'biceps',
    'triceps': 'triceps',
    'avant-bras': 'forearms',
    'quadriceps': 'quadriceps',
    'ischio-jambiers': 'hamstrings',
    'mollets': 'calves',
    'fessiers': 'glutes',
    'adducteurs': 'adductors',
    'abducteurs': 'abductors',
    'bas du dos': 'lower back',
    'milieu du dos': 'middle back',
    'lats': 'lats',
    'rhomboides': 'rhomboides',
    'haut du corps': 'upper body',
    'trapezes': 'trapezes',
    'trapezes': 'traps',
    'lombaires': 'lumbar',
    'pectoraux': 'pectorals',
    'abdominaux': 'abdominals',
    'obliques': 'obliques',
    'muscles de la ceinture abdominale': 'abdominal belt muscles',
    'deltoides': 'deltoides',
    'sous-scapulaires': 'sub-scapular',
    'cou': 'neck'
}

def extract_muscles_and_time(user_question):
    # Détecter les muscles en anglais ou en français
    muscle_keywords_fr = '|'.join(muscle_translation.keys())  # mots clés en français
    muscle_keywords_en = '|'.join(muscle_translation.values())  # mots clés en anglais

    muscle_pattern_fr = re.compile(r'\b(' + muscle_keywords_fr + r')\b', re.IGNORECASE)
    muscle_pattern_en = re.compile(r'\b(' + muscle_keywords_en + r')\b', re.IGNORECASE)

    muscles_found_fr = re.findall(muscle_pattern_fr, user_question)
    muscles_found_en = re.findall(muscle_pattern_en, user_question)

    # Conserver les noms originaux des muscles pour l'affichage
    muscles_original = muscles_found_fr + muscles_found_en

    # Convertir les muscles trouvés en français en anglais pour l'API
    muscles_translated = [muscle_translation[muscle.lower()] for muscle in muscles_found_fr]
    muscles_for_api = muscles_found_en + muscles_translated

    # Supprimer les doublons dans les listes
    muscles_original = list(set(muscles_original))
    muscles_for_api = list(set(muscles_for_api))

    # Détection du temps disponible
    time_pattern = re.compile(r'(\d+)\s*(mn|minutes|min)', re.IGNORECASE)
    time_match = re.search(time_pattern, user_question)
    time_available = int(time_match.group(1)) if time_match else None

    return muscles_original, muscles_for_api, time_available



#Monthly workout
def format_workout_plan(monthly_workout, language):
    formatted_plan = ""

    if language == 'english':
        week_count = 1
        day_in_week = 1
        previous_day = 0  # Initialisation pour le suivi du dernier jour traité
        formatted_plan += f"Week {week_count}:<br>"

        for day in sorted(monthly_workout.keys(), key=lambda x: int(x.split()[1])):  # Tri des jours par numéro
            current_day = int(day.split()[1])

            # Détecter une rupture dans la séquence des jours ou après 7 jours consécutifs
            if current_day - previous_day > 1 or day_in_week > 7:
                week_count += 1
                day_in_week = 1  # Réinitialiser le compteur de jours pour la nouvelle semaine
                formatted_plan += f"<br>Week {week_count}:<br>"

            formatted_plan += f"Day {day_in_week}:<br>"
            workouts = monthly_workout[day]
            for section, exercises in workouts.items():
                formatted_plan += f" - {section}:<br>"
                for exercise in exercises:
                    formatted_plan += f"   - {exercise['name']}<br>"

            previous_day = current_day  # Mettre à jour le dernier jour traité
            day_in_week += 1  # Incrémenter le compteur de jours de la semaine

    elif language == 'french':
        semaine_count = 1
        jour_dans_semaine = 1
        previous_day = 0  # Initialisation pour le suivi du dernier jour traité
        formatted_plan += f"Semaine {semaine_count}:<br>"

        for day in sorted(monthly_workout.keys(), key=lambda x: int(x.split()[1])):  # Tri des jours par numéro
            current_day = int(day.split()[1])

            # Détecter une rupture dans la séquence des jours ou après 7 jours consécutifs
            if current_day - previous_day > 1 or jour_dans_semaine > 7:
                semaine_count += 1
                jour_dans_semaine = 1  # Réinitialiser le compteur de jours pour la nouvelle semaine
                formatted_plan += f"<br>Semaine {semaine_count}:<br>"

            formatted_plan += f"Jour {jour_dans_semaine}:<br>"
            workouts = monthly_workout[day]
            for section, exercises in workouts.items():
                formatted_plan += f" - {section}:<br>"
                for exercise in exercises:
                    formatted_plan += f"   - {exercise['name']}<br>"

            previous_day = current_day  # Mettre à jour le dernier jour traité
            jour_dans_semaine += 1  # Incrémenter le compteur de jours de la semaine

    return formatted_plan
#Daily Workout
def format_daily_workout(daily_workout, language, time_available, muscles_original, muscles_for_api):
    formatted_plan = ""

    if language == 'english':
        # Use the translated English muscle names (muscles_for_api) for display
        muscle_list = ' and '.join(muscles_for_api)

        formatted_plan += f"Your Daily Workout Plan for {time_available} minutes targeting {muscle_list}:<br>"

        for section, exercises in daily_workout.items():
            formatted_plan += f" - {section}:<br>"
            for exercise in exercises:
                exercise_name = exercise['name']
                muscles_involved = exercise.get('muscles_involved', [])
                muscles_involved_str = ', '.join(muscles_involved)
                formatted_plan += f"   - {exercise_name} (Muscles involved: {muscles_involved_str})<br>"

    elif language == 'french':
        muscle_list = ' et '.join(muscles_original)  # Use original French muscle names for display
        formatted_plan += f"Votre plan d'entraînement quotidien de {time_available} minutes pour {muscle_list}:<br>"

        for section, exercises in daily_workout.items():
            formatted_plan += f" - {section}:<br>"
            for exercise in exercises:
                exercise_name = exercise['name']
                muscles_involved = exercise.get('muscles_involved', [])
                muscles_involved_str = ', '.join(muscles_involved)
                formatted_plan += f"   - {exercise_name} (Muscles impliqués : {muscles_involved_str})<br>"

    return formatted_plan
# Fonction pour vider la mémoire GPU
def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()

# Générer une réponse avec mBART (Français)
def generate_response_mbart(question, max_input_length=132, max_output_length=1024):
    try:
        inputs = tokenizer_fr([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)
        outputs = model_fr.generate(
            inputs["input_ids"],
            max_length=max_output_length,
            num_beams=8,
            no_repeat_ngram_size=4,
            length_penalty=1.8,
            repetition_penalty=2.5,
            temperature=0.2,
            top_p=0.9,
            top_k=50
        )
        return tokenizer_fr.decode(outputs[0], skip_special_tokens=True)

    except RuntimeError as e:
        if 'CUDA out of memory' in str(e):
            print("Erreur: Mémoire GPU saturée, tentative de libération de mémoire...")
            clear_memory()  # Libérer la mémoire GPU
            return "La mémoire est saturée, veuillez réessayer plus tard."
        else:
            raise e  # Remonter toute autre erreur

# Générer une réponse avec BART (Anglais)
def generate_response_bart(question, max_input_length=132, max_output_length=1024):
    try:
        inputs = tokenizer_en([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)
        outputs = model_en.generate(
            inputs["input_ids"],
            max_length=max_output_length,
            num_beams=8,
            no_repeat_ngram_size=4,
            length_penalty=1.8,
            repetition_penalty=2.5,
            temperature=0.2,
            top_p=0.9,
            top_k=50
        )
        return tokenizer_en.decode(outputs[0], skip_special_tokens=True)

    except RuntimeError as e:
        if 'CUDA out of memory' in str(e):
            print("Erreur: Mémoire GPU saturée, tentative de libération de mémoire...")
            clear_memory()  # Libérer la mémoire GPU
            return "La mémoire est saturée, veuillez réessayer plus tard."
        else:
            raise e  # Remonter toute autre erreur
def chatbot_response(user_question):
    user_id = 1
    user_language = detect_language(user_question)  # Détecter la langue de l'utilisateur
    user_question_lower = user_question.lower()

    # Vérifier si l'utilisateur demande un plan d'entraînement quotidien (daily workout)
    if any(keyword in user_question_lower for keyword in ["daily workout", "entraînement quotidien", "give me a workout", "donner moi un entrainement"]):

        # Extraire les muscles et le temps disponibles
        muscles_original, muscles_for_api, time_available = extract_muscles_and_time(user_question)

        if len(muscles_for_api) > 2:
            return "You can only select up to 2 muscles for a daily workout plan." if user_language == 'english' else "Vous ne pouvez sélectionner que jusqu'à 2 muscles pour un plan d'entraînement quotidien."

        if not time_available:
            return "Please specify a time for the workout." if user_language == 'english' else "Veuillez spécifier un temps pour l'entraînement."

        # Vérifier si le temps et les muscles sont bien définis
        if muscles_for_api and time_available:
            try:
                # Générer un plan d'entraînement quotidien
                daily_workout = call_daily_workout_api(user_id, muscles_for_api, time_available, user_language)
                if daily_workout:
                    # Formatage de la réponse
                    formatted_plan = format_daily_workout(daily_workout, user_language, time_available, muscles_original, muscles_for_api)
                    return formatted_plan
                else:
                    return "Unable to generate a daily workout plan ,please try again later." if user_language == 'english' else "Impossible de générer un programme d'entraînement quotidien. Veuillez réessayer plus tard."
            except Exception as e:
                return f"Error calling the API: {e}" if user_language == 'english' else f"Erreur lors de l'appel de l'API: {e}"

    # Vérifier si l'utilisateur demande un programme mensuel
    elif any(keyword in user_question_lower for keyword in ["monthly program", "monthly workout", "programme mensuel", "entraînement mensuel"]):
        try:
                monthly_workout = call_monthly_workout_api(user_id, user_language)
                if monthly_workout:
                    return f"Voici votre programme d'entraînement mensuel:<br>{format_workout_plan(monthly_workout, user_language)}" if user_language == 'french' else f"Here is your monthly workout program:<br>{format_workout_plan(monthly_workout, user_language)}"
                else:
                    return "Impossible de générer un programme d'entraînement mensuel.Veuillez réessayer plus tard." if user_language == 'french' else "Unable to generate a monthly workout program,please try again later."
        except Exception as e:
                return f"Erreur lors de l'appel de l'API: {e}" if user_language == 'french' else f"Error calling the API: {e}"


    embeddings, questions, responses = (french_embeddings, french_questions, french_responses) if user_language == 'french' else (english_embeddings, english_questions, english_responses)
    best_question, best_response = find_similar_question_and_response(user_question, embeddings, questions, responses, top_k=5)

    if best_question:
            return best_response
    else:
          if user_language == 'french':
            return generate_response_mbart(user_question)
          else:
            return generate_response_bart(user_question)
# Lancer ngrok sur le port 5000
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Public URL: NgrokTunnel: "https://089f-34-105-80-245.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 18:42:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 18:42:15] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 18:42:42] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 18:44:25] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 18:44:56] "POST /get_response HTTP/1.1" 200 -


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
from langdetect import detect

# Charger les modèles Sentence-BERT multilingue et Cross-Encoder
bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Fonction pour détecter la langue du texte
def detect_language(text):
    try:
        language = detect(text)
        return 'french' if language == 'fr' else 'english'
    except:
        return 'french'

# Fonction pour prétraiter le texte en fonction de la langue
def preprocess_text(text, language):
    text = text.lower()
    if language == 'french':
        text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('à', 'a')
        text = text.replace('ç', 'c').replace('ô', 'o').replace('û', 'u').replace('î', 'i')
    return text

# Fonction pour calculer le pourcentage de mots communs
def calculate_word_overlap(user_question, best_question):
    user_words = set(user_question.split())
    best_words = set(best_question.split())
    common_words = user_words.intersection(best_words)

    # Calculer le pourcentage de correspondance
    overlap_percentage = len(common_words) / len(user_words) * 100
    return overlap_percentage

# Lire les datasets français et anglais
french_data = pd.read_csv('/content/french_dataset.csv')
english_data = pd.read_csv('/content/english_dataset.csv')

# Nettoyer les données
french_data['Question'] = french_data['Question'].apply(lambda x: preprocess_text(x, 'french'))
english_data['Question'] = english_data['Question'].apply(lambda x: preprocess_text(x, 'english'))

# Encodage des questions avec le Bi-Encoder
french_embeddings = bi_encoder.encode(french_data['Question'].tolist(), convert_to_tensor=True)
english_embeddings = bi_encoder.encode(english_data['Question'].tolist(), convert_to_tensor=True)

# Fonction pour trouver les questions similaires et vérifier le taux de précision
def find_similar_question_and_response(user_question, embeddings, questions, responses, top_k=5, threshold=75):
    # Prétraiter la question de l'utilisateur
    user_question_preprocessed = preprocess_text(user_question, detect_language(user_question))

    # Encoder la question de l'utilisateur avec le Bi-Encoder
    user_embedding = bi_encoder.encode(user_question_preprocessed, convert_to_tensor=True)

    # Calculer la similarité cosinus entre l'utilisateur et les questions du dataset
    cosine_scores = util.pytorch_cos_sim(user_embedding, embeddings)[0]

    # Sélectionner les top_k questions les plus similaires avec le Bi-Encoder
    top_k_indices = torch.topk(cosine_scores, k=top_k).indices

    # Extraire les questions similaires et leurs réponses associées, et les scores de similarité Bi-Encoder
    similar_questions = [(questions[idx.item()], responses[idx.item()], cosine_scores[idx].item()) for idx in top_k_indices]

    # Afficher les similarités Bi-Encoder pour les top-k questions
    print("\nTop-k questions similaires selon le Bi-Encoder :")
    for i, (question, response, score) in enumerate(similar_questions):
        print(f"{i+1}. Question: {question} - Score de similarité Bi-Encoder: {score:.4f}")

    # Vérifier si l'une des similarités Bi-Encoder est supérieure ou égale à 0.96
    for question, response, score in similar_questions:
        if score >= 0.95:
            print(f"Correspondance directe trouvée avec une similarité de {score:.4f} (>= 0.96)")
            return question, response

    # Si aucune correspondance n'est trouvée avec un score >= 0.96, utiliser le Cross-Encoder
    pairs = [[user_question_preprocessed, question] for question, _, _ in similar_questions]
    cross_scores = cross_encoder.predict(pairs)

    # Afficher les similarités Cross-Encoder pour les top-k questions
    print("\nScores de similarité Cross-Encoder pour les questions similaires :")
    for i, (question, _, score) in enumerate(similar_questions):
        print(f"{i+1}. Question: {question} - Score de similarité Cross-Encoder: {cross_scores[i]:.4f}")

    # Trouver l'index de la meilleure question en fonction du score du Cross-Encoder
    best_match_idx = cross_scores.argmax()
    best_question, best_response = similar_questions[best_match_idx][:2]

    # Calculer le pourcentage de mots communs
    word_overlap = calculate_word_overlap(user_question_preprocessed, best_question)

    # Afficher le taux de précision (pourcentage de mots communs)
    print(f"\nTaux de précision (pourcentage de mots communs) : {word_overlap:.2f}%")

    # Vérifier si le taux de précision est supérieur ou égal au seuil
    if word_overlap >= threshold:
        return best_question, best_response
    else:
        return None, "Désolé, je n'ai pas trouvé une correspondance précise. Voici une suggestion générale : [fallback response]"

# Boucle pour saisir des questions dans le terminal
while True:
    user_question = input("Entrez votre question (ou tapez 'exit' pour quitter) : ")
    if user_question.lower() == 'exit':
        break

    user_language = detect_language(user_question)
    embeddings, questions, responses = (french_embeddings, french_data['Question'].tolist(), french_data['Réponse'].tolist()) if user_language == 'french' else (english_embeddings, english_data['Question'].tolist(), english_data['Réponse'].tolist())

    best_question, best_response = find_similar_question_and_response(user_question, embeddings, questions, responses, top_k=5)

    if best_response:
        print(f"\nMeilleure correspondance : {best_question}\nRéponse : {best_response}")
    else:
        print(best_response)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Entrez votre question (ou tapez 'exit' pour quitter) : comment on peux faire clean and jerk?

Top-k questions similaires selon le Bi-Encoder :
1. Question: comment faire le jerk (jete) apres le clean ? - Score de similarité Bi-Encoder: 0.8858
2. Question: comment apprendre le clean and jerk ? - Score de similarité Bi-Encoder: 0.8378
3. Question: comment apprendre le clean and jerk (epaule-jete) ? - Score de similarité Bi-Encoder: 0.7999
4. Question: quels sont les avantages du clean and jerk ? - Score de similarité Bi-Encoder: 0.7203
5. Question: comment determiner l’intensite de travail pour le clean and jerk ? - Score de similarité Bi-Encoder: 0.7054

Scores de similarité Cross-Encoder pour les questions similaires :
1. Question: comment faire le jerk (jete) apres le clean ? - Score de similarité Cross-Encoder: 5.6584
2. Question: comment apprendre le clean and jerk ? - Score de similarité Cross-Encoder: 2.4552
3. Question: comment apprendre le clean and jerk (epaule-jete) ? - Score 

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
english_data

,Question,Réponse,question_length,reponse_length,input_text,target_text
0,"why target 10,000 steps every day?","the 10,000 -step target is often recommended t...",34,185,"why target 10,000 steps every day?","the 10,000 -step target is often recommended t..."
1,what are the real health benefits?,"walking helps improve physical condition, redu...",34,147,what are the real health benefits?,"walking helps improve physical condition, redu..."
2,how to make chocolate mousse with only two ing...,it is possible to make chocolate mousse using ...,55,184,how to make chocolate mousse with only two ing...,it is possible to make chocolate mousse using ...
3,what are the ingredients necessary for chocola...,"to prepare the chocolate mousse with water, yo...",65,167,what are the ingredients necessary for chocola...,"to prepare the chocolate mousse with water, yo..."
4,how to prepare fast slimming recipes with a fr...,air fryer allows you to prepare tasty and bala...,55,147,how to prepare fast slimming recipes with a fr...,air fryer allows you to prepare tasty and bala...
...,...,...,...,...,...,...
9534,what are the benefits of band chest fly?,the benefits are: bands provide dynamic tensio...,40,279,what are the benefits of band chest fly?,the benefits are: bands provide dynamic tensio...
9535,what are the benefits of band overhead squat?,the benefits are: develops mobility and stabil...,45,251,what are the benefits of band overhead squat?,the benefits are: develops mobility and stabil...
9536,what are the benefits of band standing concent...,the benefits are: builds stronger and more mus...,58,242,what are the benefits of band standing concent...,the benefits are: builds stronger and more mus...
9537,what are the benefits of band seated row?,the benefits are: the band provides dynamic te...,41,281,what are the benefits of band seated row?,the benefits are: the band provides dynamic te...


In [ ]:
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import Dataset
from transformers import AdamW, get_cosine_schedule_with_warmup
import pandas as pd
import os
from google.colab import drive  # Import for Google Drive integration

# Mount Google Drive

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load BART-large model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# Load English dataset
english_data = pd.read_csv('/content/english_dataset.csv')

# Add columns `input_text` and `target_text`
english_data['input_text'] = english_data['Question']
english_data['target_text'] = english_data['Réponse']

# Drop missing values
english_data = english_data.dropna(subset=['input_text', 'target_text']).reset_index(drop=True)

# Convert to Hugging Face Dataset
english_dataset = Dataset.from_pandas(english_data[['input_text', 'target_text']])

# Split into train/test sets
english_train_test_split = english_dataset.train_test_split(test_size=0.1)

# Preprocessing function for English data
def preprocess_function(examples,  max_input_length=132, max_target_length=1978):
    inputs = examples['input_text']
    targets = examples['target_text']

    # Tokenize inputs with padding and truncation
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="longest")

    # Tokenize targets (output) with padding and truncation
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="longest").input_ids

    # Replace pad token ID in labels with -100 (this ignores them in the loss calculation)
    labels = torch.tensor(labels)
    labels = torch.where(labels == tokenizer.pad_token_id, -100, labels)
    model_inputs["labels"] = labels

    return model_inputs


# Tokenize the English dataset
print("Tokenizing English dataset...")
tokenized_train_en = english_train_test_split['train'].map(
    lambda x: preprocess_function(x), batched=True, remove_columns=['input_text', 'target_text']
)
tokenized_eval_en = english_train_test_split['test'].map(
    lambda x: preprocess_function(x), batched=True, remove_columns=['input_text', 'target_text']
)

# Data collator with dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest", return_tensors="pt")

optimizer = AdamW(model.parameters(), lr=1e-5)

# Faster cosine scheduler
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=len(tokenized_train_en)
)

# Training arguments with increased batch size and dynamic padding
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,  # Increased batch size
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Number of epochs
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=8,  # Adjusted for larger batch size
    load_best_model_at_end=True,
    gradient_checkpointing=True,  # Save memory with gradient checkpointing
    fp16=True if device == "cuda" else False,  # Mixed precision training
    dataloader_num_workers=0,  # Disable multi-threading if necessary
    remove_unused_columns=False,
    save_total_limit=3,
    max_grad_norm=1.0,  # Ensure gradient clipping
)

# Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_en,
    eval_dataset=tokenized_eval_en,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),  # Pass optimizer and scheduler
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],  # Early stopping callback
)

# Train the model
print("Starting training...")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results: {eval_results}")

# Save the model to Google Drive
save_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_bart_final_10epoch'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenizing English dataset...


Map:   0%|          | 0/8585 [00:00<?, ? examples/s]

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,3.040900,2.748919
1,2.810700,2.596151
2,2.668100,2.515453
4,2.544500,2.410342
5,2.485700,2.371230
6,2.412600,2.343005
8,2.286500,2.297668
9,2.205300,2.284458


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarnin

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Evaluation results: {'eval_loss': 2.2844583988189697, 'eval_runtime': 17.0477, 'eval_samples_per_second': 55.961, 'eval_steps_per_second': 7.039, 'epoch': 9.981378026070763}
Model saved to /content/drive/MyDrive/Colab Notebooks/mon_modele_bart_final_10epoch


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_bart_final_10epoch'
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
def generate_response(question, max_input_length=132, max_output_length=1024):
    # Tokenize input question
    inputs = tokenizer([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)

    outputs = model.generate(
    inputs["input_ids"],
    max_length=max_output_length,  # Set max output length, e.g., 1024
    num_beams=8,
    no_repeat_ngram_size=4,  # Increase to reduce repetition
    length_penalty=1.8,  # Higher penalty for longer responses
    repetition_penalty=2.5,  # Stronger penalty for repeated words
    temperature=0.2,  # Lower temperature for more deterministic responses
    top_p=0.9,  # Focus on top 90% most likely words
    top_k=50
)
    # Decode the output and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model with a new question
question = "how to make the tiger posture?"
response = generate_response(question)
print(f"Question: {question}")
print(f"Response: {response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: how to make the tiger posture?
Response: 1: enter the posture of the tiger with your feet apart from the width of the shoulders.2: place your hands behind your head and fold your knees at 90 degrees.3: take a deep breath, then exhale by bringing your chest back to the starting position.4: repeat this movement until you have made the desired number of rehearsals.5: return slowly to the initial position.


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Charger nltk (si nécessaire pour les corpus)
nltk.download('punkt')

# Charger les données en anglais ou français selon le modèle utilisé
# Pour cet exemple, je vais utiliser les données anglaises, mais vous pouvez l'adapter pour le modèle mBART aussi
english_data = pd.read_csv('/content/english_dataset.csv')

# Ajouter les colonnes `input_text` et `target_text`
english_data['input_text'] = english_data['Question']
english_data['target_text'] = english_data['Réponse']

# Supprimer les valeurs manquantes
english_data = english_data.dropna(subset=['input_text', 'target_text']).reset_index(drop=True)

# Convertir en dataset Hugging Face
english_dataset = Dataset.from_pandas(english_data[['input_text', 'target_text']])

# Charger le modèle et le tokenizer BART
model_path_en = '/content/drive/MyDrive/Colab Notebooks/mon_modele_bart_final_10epoch'
model_en = BartForConditionalGeneration.from_pretrained(model_path_en)
tokenizer_en = BartTokenizer.from_pretrained("facebook/bart-large")

# Diviser en ensembles d'entraînement et de test
english_train_test_split = english_dataset.train_test_split(test_size=0.1)

# Fonction de prétraitement pour les données
def preprocess_function_en(examples, max_input_length=132, max_target_length=1978):
    inputs = examples['input_text']
    targets = examples['target_text']

    # Tokenizer les entrées (questions)
    model_inputs = tokenizer_en(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Tokenizer les sorties (réponses)
    labels = tokenizer_en(targets, max_length=max_target_length, truncation=True, padding="max_length").input_ids

    # Remplacer les tokens de padding par -100 dans les labels
    labels = [label if label != tokenizer_en.pad_token_id else -100 for label in labels]

    model_inputs["labels"] = labels

    return model_inputs

# Tokenizer le dataset d'évaluation anglais
print("Tokenizing English dataset...")

tokenized_eval_en = english_train_test_split['test'].map(
    lambda x: preprocess_function_en(x), batched=True, remove_columns=['input_text', 'target_text']
)

# Définir l'appareil (GPU si disponible, sinon CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_en.to(device)

# Fonction pour générer une réponse à partir d'une question
def generate_response_en(question, max_input_length=132, max_output_length=1024):
    inputs = tokenizer_en([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)

    outputs = model_en.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=8,
        no_repeat_ngram_size=4,
        length_penalty=1.8,
        repetition_penalty=2.5,
        temperature=0.2,
        top_p=0.9,
        top_k=50
    )

    return tokenizer_en.decode(outputs[0], skip_special_tokens=True)

# Fonction pour calculer le score BLEU
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Fonction pour calculer le score BLEU avec lissage
def calculate_bleu_score(reference, generated):
    reference = [nltk.word_tokenize(reference.lower())]  # Tokeniser la référence
    generated = nltk.word_tokenize(generated.lower())  # Tokeniser la réponse générée
    # Appliquer la fonction de lissage
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference, generated, smoothing_function=smoothie)
    return bleu_score


# Fonction pour évaluer le modèle avec ROUGE et BLEU
def evaluate_model_en(dataset, max_samples=100):
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0, "bleu": 0}
    num_samples = min(len(dataset), max_samples)

    for idx, data in enumerate(dataset):
        if idx >= max_samples:
            break

        question = data["input_text"]
        reference = data["target_text"]

        # Générer la réponse du modèle
        generated_response = generate_response_en(question)

        # Calculer les scores ROUGE
        rouge_score = rouge_scorer_instance.score(reference, generated_response)

        # Calculer le score BLEU
        bleu_score = calculate_bleu_score(reference, generated_response)

        # Accumuler les scores
        for key in rouge_score:
            scores[key] += rouge_score[key].fmeasure
        scores["bleu"] += bleu_score

    # Moyenne des scores
    for key in scores:
        scores[key] /= num_samples

    return scores

# Exécuter l'évaluation pour le modèle anglais
print("Evaluating the English model on the test set with BLEU and ROUGE...")

evaluation_scores = evaluate_model_en(english_train_test_split['test'])

# Afficher les résultats
print("Scores d'évaluation (modèle anglais) :")
for key, score in evaluation_scores.items():
    print(f"{key}: {score:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenizing English dataset...


Map:   0%|          | 0/954 [00:00<?, ? examples/s]

Evaluating the English model on the test set with BLEU and ROUGE...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Scores d'évaluation (modèle anglais) :
rouge1: 0.3598
rouge2: 0.1264
rougeL: 0.2591
bleu: 0.0821


In [ ]:
# Test the model with a new question
question = "what is hiit?"
response = generate_response(question)
print(f"Question: {question}")
print(f"Response: {response}")

Question: what is hiit?
Response: hiit, or high intensity interval training, is an intense physical activity that mainly targets the muscles of the legs and glutes.it can be practiced at any time of the day for various reasons, such as weight loss, muscle strengthening, or to improve endurance.


In [ ]:
# Test the model with a new question
question = "give me advice for extensions of lyte triceps with dumbbells"
response = generate_response(question)
print(f"Question: {question}")
print(f"Response: {response}")

Question: give me advice for extensions of lyte triceps with dumbbells
Response: for the exercise extensions of lyte triceps with dumbbells, do not lock your elbows at the top of the movement.keep a slight bending of the elbows to maintain tension on the triceps.do not let the dumbbells touch each other throughout the movement.if you feel pain in the elbow joint, start with light loads and gradually increase the load as you get used to it.


french mbrat model


In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5bce7ba54ddba9f3cb2f3a32e37c5afff42a885a8447417024bc9800ccb7e739
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import MBartForConditionalGeneration, MBartTokenizer
from transformers import AdamW, get_cosine_schedule_with_warmup
import pandas as pd
import os
from google.colab import drive  # Import for Google Drive integration
from datasets import Dataset
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Check if CUDA is available and set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


# Charger le modèle mBART-large-cc25 et le tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="fr_XX", tgt_lang="fr_XX")
# Load French dataset
french_data = pd.read_csv('/content/french_dataset.csv')

# Add columns `input_text` and `target_text`
french_data['input_text'] = french_data['Question']
french_data['target_text'] = french_data['Réponse']

# Drop missing values
french_data = french_data.dropna(subset=['input_text', 'target_text']).reset_index(drop=True)

# Convert to Hugging Face Dataset
french_dataset = Dataset.from_pandas(french_data[['input_text', 'target_text']])
# Preprocessing function for French data
def preprocess_function_fr(examples, max_input_length=132, max_target_length=1978):
    inputs = examples['input_text']
    targets = examples['target_text']

    # Tokenize inputs with padding and truncation
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="longest")

    # Tokenize targets (output) with padding and truncation
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="longest").input_ids

    # Replace pad token ID in labels with -100 (this ignores them in the loss calculation)
    labels = torch.tensor(labels)
    labels = torch.where(labels == tokenizer.pad_token_id, -100, labels)
    model_inputs["labels"] = labels

    return model_inputs
# Split into train/test sets
french_train_test_split = french_dataset.train_test_split(test_size=0.1)

# Tokenize the French dataset
print("Tokenizing French dataset...")
tokenized_train_fr = french_train_test_split['train'].map(
    lambda x: preprocess_function_fr(x), batched=True, remove_columns=['input_text', 'target_text']
)
tokenized_eval_fr = french_train_test_split['test'].map(
    lambda x: preprocess_function_fr(x), batched=True, remove_columns=['input_text', 'target_text']
)

# Data collator with dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest", return_tensors="pt")

# Define training arguments for French model with optimizations
training_args_fr = TrainingArguments(
    output_dir="./results_french",
    evaluation_strategy="epoch",  # Sauvegarde et évaluation à chaque époque
    save_strategy="epoch",        # Sauvegarde à la fin de chaque époque
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=10,
    save_total_limit=2,
    fp16=True,
    logging_dir='./logs_fr',
    logging_steps=10,
    load_best_model_at_end=True,  # Charger le meilleur modèle
    metric_for_best_model="loss",
    label_smoothing_factor=0.1,
)


# Calculate the number of training steps
num_training_steps = (len(tokenized_train_fr) // (training_args_fr.per_device_train_batch_size * training_args_fr.gradient_accumulation_steps)) * training_args_fr.num_train_epochs

# Optimizer and scheduler setup
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args_fr,
    train_dataset=tokenized_train_fr,
    eval_dataset=tokenized_eval_fr,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

# Train the model
print("Starting training...")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results: {eval_results}")

# Save the model to Google Drive
save_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_mbart_final_10'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

...

Starting training...


Epoch,Training Loss,Validation Loss
0,3.742800,3.577566
1,3.514500,3.379165
2,3.294400,3.292774
3,3.220300,3.248920
4,3.146700,3.215569
5,3.101300,3.197827
6,3.041900,3.189848
7,3.032500,3.187016
8,2.968900,3.184880
9,3.033400,3.185125


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) 

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}


Evaluation results: {'eval_loss': 3.184880018234253, 'eval_runtime': 81.805, 'eval_samples_per_second': 11.283, 'eval_steps_per_second': 11.283, 'epoch': 9.996388587937883}
Model saved to /content/drive/MyDrive/Colab Notebooks/mon_modele_mbart_final_10


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from rouge_score import rouge_scorer

# Charger les données
french_data = pd.read_csv('/content/french_dataset.csv')

# Ajouter les colonnes `input_text` et `target_text`
french_data['input_text'] = french_data['Question']
french_data['target_text'] = french_data['Réponse']

# Supprimer les valeurs manquantes
french_data = french_data.dropna(subset=['input_text', 'target_text']).reset_index(drop=True)

# Convertir en dataset Hugging Face
french_dataset = Dataset.from_pandas(french_data[['input_text', 'target_text']])

# Charger le tokenizer MBart pour le français
tokenizer_fr = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="fr_XX", tgt_lang="fr_XX")

# Fonction de prétraitement pour les données en français
def preprocess_function_fr(examples, max_input_length=132, max_target_length=1978):
    inputs = examples['input_text']
    targets = examples['target_text']

    # Tokenizer les entrées (questions)
    model_inputs = tokenizer_fr(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Tokenizer les sorties (réponses)
    labels = tokenizer_fr(targets, max_length=max_target_length, truncation=True, padding="max_length").input_ids

    # Remplacer les tokens de padding par -100 dans les labels
    labels = [label if label != tokenizer_fr.pad_token_id else -100 for label in labels]

    model_inputs["labels"] = labels

    return model_inputs

# Diviser en ensembles d'entraînement et de test
french_train_test_split = french_dataset.train_test_split(test_size=0.1)

# Tokenizer le dataset d'évaluation français
print("Tokenizing French dataset...")

tokenized_eval_fr = french_train_test_split['test'].map(
    lambda x: preprocess_function_fr(x), batched=True, remove_columns=['input_text', 'target_text']
)

# Charger le modèle MBart que vous avez déjà fine-tuné pour le français
model_path_fr = '/content/drive/MyDrive/Colab Notebooks/mon_modele_mbart_final_3epoch'
model_fr = MBartForConditionalGeneration.from_pretrained(model_path_fr)

# Définir l'appareil (GPU si disponible, sinon CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_fr.to(device)

# Fonction pour générer une réponse à partir d'une question (Modèle pour le français)
def generate_response_fr(question, max_input_length=132, max_output_length=1024):
    # Tokenizer la question d'entrée
    inputs = tokenizer_fr([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)

    # Générer une réponse avec le modèle (modèle français)
    outputs = model_fr.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=8,
        no_repeat_ngram_size=4,
        length_penalty=1.8,
        repetition_penalty=2.5,
        temperature=0.2,
        top_p=0.9,
        top_k=50
    )

    # Décoder la réponse générée et retourner le texte
    return tokenizer_fr.decode(outputs[0], skip_special_tokens=True)

# Fonction pour évaluer le modèle avec ROUGE (Modèle pour le français)
def evaluate_model_fr(dataset, max_samples=100):
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    num_samples = min(len(dataset), max_samples)  # Limiter le nombre d'échantillons si nécessaire

    for idx, data in enumerate(dataset):
        if idx >= max_samples:
            break

        question = data["input_text"]
        reference = data["target_text"]

        # Générer la réponse du modèle en français
        generated_response = generate_response_fr(question)

        # Calculer les scores ROUGE
        score = rouge_scorer_instance.score(reference, generated_response)

        # Accumuler les scores
        for key in scores:
            scores[key] += score[key].fmeasure

    # Moyenne des scores
    for key in scores:
        scores[key] /= num_samples

    return scores

# Exécuter l'évaluation pour le modèle français
print("Evaluating the French model on the test set...")

rouge_scores_fr = evaluate_model_fr(french_train_test_split['test'])

# Afficher les résultats
print("Scores ROUGE (modèle français):")
for key, score in rouge_scores_fr.items():
    print(f"{key}: {score:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

...

Tokenizing French dataset...


Map:   0%|          | 0/923 [00:00<?, ? examples/s]

Evaluating the French model on the test set...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Scores ROUGE (modèle français):
rouge1: 0.3666
rouge2: 0.1459
rougeL: 0.2578


In [ ]:
!pip install evaluate


In [ ]:
import evaluate
import numpy as np

# Charger les métriques BLEU et ROUGE
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")

# Fonction pour générer des prédictions
def generate_predictions(dataset, model, tokenizer):
    predictions = []
    references = []

    for example in dataset:
        input_text = example['input_text']
        target_text = example['target_text']

        # Tokenisation des entrées
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).input_ids.to(device)

        # Génération avec le modèle
        output_tokens = model.generate(inputs, max_length=1978, num_beams=4)
        predicted_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        # Ajouter les prédictions et références
        predictions.append(predicted_text)
        references.append([target_text])  # BLEU attend une liste de listes pour les références

    return predictions, references

# Générer les prédictions pour l'ensemble de test
print("Generating predictions...")
predictions, references = generate_predictions(tokenized_eval_fr, model, tokenizer)

# Calculer le score BLEU
bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score['score']}")

# Calculer les scores ROUGE
rouge_score = rouge_metric.compute(predictions=predictions, references=references, rouge_types=["rouge1", "rouge2", "rougeL"])
print(f"ROUGE-1 score: {rouge_score['rouge1'].mid.fmeasure}")
print(f"ROUGE-2 score: {rouge_score['rouge2'].mid.fmeasure}")
print(f"ROUGE-L score: {rouge_score['rougeL'].mid.fmeasure}")


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Chemin vers le modèle sauvegardé
model_path = '/content/drive/MyDrive/Colab Notebooks/mon_modele_mbart_final_3epoch'

# Charger le tokenizer et le modèle sauvegardés
tokenizer_fr = MBart50TokenizerFast.from_pretrained(model_path, src_lang="fr_XX", tgt_lang="fr_XX")
model_fr = MBartForConditionalGeneration.from_pretrained(model_path)

# Définir l'appareil (GPU si disponible, sinon CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_fr.to(device)

# Fonction pour générer une réponse à partir d'une question
def generate_response(question, max_input_length=132, max_output_length=1024):
    # Tokenizer la question d'entrée
    inputs = tokenizer_fr([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)

    # Générer une réponse avec le modèle
    outputs = model_fr.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=8,
        no_repeat_ngram_size=4,
        length_penalty=1.8,
        repetition_penalty=2.5,
        temperature=0.2,
        top_p=0.9,
        top_k=50
    )

    # Décoder la réponse générée et retourner le texte
    return tokenizer_fr.decode(outputs[0], skip_special_tokens=True)

# Test du modèle avec une nouvelle question
question = "comment faire clean and jerk?"
response = generate_response(question)

# Afficher la question et la réponse
print(f"Question: {question}")
print(f"Response: {response}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: comment faire clean and jerk?
Response: pour faire clean and jerk, allongez-vous sur le sol, les pieds écartés de la largeur des hanches et les genoux légèrement fléchis. saisissez les poignées avec une prise en pronation (paumes vers l'intérieur). descendez jusqu'à ce que vos jambes soient parallèles au sol, puis revenez à la position de départ. répétez le mouvement jusqu'à effectuer le nombre de répétitions souhaité.


In [ ]:
def generate_response(question, max_input_length=132, max_output_length=512):  # Adjusted max_output_length
    # Tokenizer la question d'entrée
    inputs = tokenizer([question], max_length=max_input_length, truncation=True, return_tensors="pt", padding="longest").to(device)

    # Générer une réponse avec le modèle
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_output_length,
        num_beams=4,  # Reduced from 8 to 4
        no_repeat_ngram_size=3,  # Reduced n-gram repetition penalty
        length_penalty=1.2,  # Reduced length penalty for shorter responses
        repetition_penalty=1.5,  # Adjusted repetition penalty
        temperature=0.7,  # Increased temperature for more diverse answers
        top_p=0.9,
        top_k=50
    )

    # Décoder la réponse générée et retourner le texte
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test du modèle avec une nouvelle question
question = "comment on peut rester hydrater?"
response = generate_response(question)

# Afficher la question et la réponse
print(f"Question: {question}")
print(f"Response: {response}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: comment on peut rester hydrater?
Response: pour rester hydraté, il est important de s'échauffer régulièrement, de consommer des aliments riches en eau, et de maintenir un mode de vie sain.


In [ ]:
question = "comment faire clean and jerk?"
response = generate_response(question)

# Afficher la question et la réponse
print(f"Question: {question}")
print(f"Response: {response}")

Question: comment faire clean and jerk?
Response: pour faire clean and jerk, allongez-vous sur le sol, les pieds écartés de la largeur des hanches, et les genoux légèrement fléchis. abaissez votre corps vers l'arrière jusqu'à ce qu'il soit parallèle au sol, puis revenez à la position de départ. répétez le mouvement jusqu'au nombre de répétitions souhaité.
